# EduHub MongoDB Project

This notebook demonstrates the implementation of MongoDB operations for the EduHub educational platform.

In [ ]:
# Import required libraries
from eduhub_queries import EduHubDB
from datetime import datetime
import pandas as pd

# Initialize database connection
db = EduHubDB()

## Part 1: Database Setup and Data Modeling

In [ ]:
# Create indexes for performance optimization
db.create_indexes()

## Part 2: Data Population

In [ ]:
# Sample users
students = [{
    "email": "student{}@example.com".format(i),
    "firstName": "Student",
    "lastName": "{}".format(i),
    "role": "student"
} for i in range(1, 11)]

instructors = [{
    "email": "instructor{}@example.com".format(i),
    "firstName": "Instructor",
    "lastName": "{}".format(i),
    "role": "instructor",
    "profile": {
        "bio": "Experienced educator in {}".format(i),
        "skills": ["Python", "Data Science", "Machine Learning"]
    }
} for i in range(1, 6)]

# Insert users
student_ids = [db.add_user(student) for student in students]
instructor_ids = [db.add_user(instructor) for instructor in instructors]

In [ ]:
# Sample courses
courses = [{
    "title": "Course {}".format(i),
    "description": "Description for course {}".format(i),
    "instructorId": instructor_ids[i % len(instructor_ids)],
    "category": "Category {}".format(i % 3 + 1),
    "level": ["beginner", "intermediate", "advanced"][(i-1) % 3],
    "price": 99.99 * i,
    "tags": ["programming", "data science", "machine learning"]
} for i in range(1, 9)]

# Insert courses
course_ids = [db.add_course(course) for course in courses]

## Part 3: Basic CRUD Operations

In [ ]:
# Create operations
# Enroll students in courses
for i, student_id in enumerate(student_ids):
    course_id = course_ids[i % len(course_ids)]
    
    db.enroll_student(student_id, course_id)

In [ ]:
# Read operations
# Get all active students
active_students = list(db.db.users.find({"role": "student", "isActive": True}))
pd.DataFrame(active_students)

In [ ]:
# Update operations
# Update a user's profile
user_id = student_ids[0]
updates = {"profile.bio": "Updated bio for student"}
db.update_user_profile(user_id, updates)

## Part 4: Advanced Queries and Aggregation

In [ ]:
# Course enrollment statistics
enrollment_stats = db.get_course_enrollment_stats()
enrollment_stats

In [ ]:
# Complex query: Courses with price between $50 and $200
affordable_courses = list(db.db.courses.find({"price": {"$gte": 50, "$lte": 200}}))
pd.DataFrame(affordable_courses)

## Part 5: Indexing and Performance

In [ ]:
# Check indexes
indexes = list(db.db.users.list_indexes())
pd.DataFrame(indexes)

## Part 6: Data Validation and Error Handling

In [ ]:

try:
    invalid_user = {
        "email": "invalid-email",  # Invalid email format
        "firstName": "Invalid",
        "lastName": "User",
        "role": "invalid_role"  # Invalid role
    }
    db.add_user(invalid_user)
except Exception as e:
    print(f"Error: {str(e)}")

## Cleanup

```python
db.close_connection()
```